작성일자: 2022-08-03<br>
작성자: 김동연

함수의 파라미터로 리스트를 받는 경우 리스트를 여러 번 순회해야 하는 경우가 있다.<br>
예를 들어 각 도시의 방문자 수를 나타내는 리스트가 있다고 생각해보자.<br>
정규화 함수를 통해 전체 방문자 중 몇 퍼센트가 각 도시를 방문했는지 알 수 있다.<br>
<br>
정규화 함수에서는 전체 방문자 수를 sum으로 구하고 각 도시의 방문자 수와 전체 방문자 수를 나눈다.

In [1]:
def normalize(numbers):
    total = sum(numbers)
    result = []
    for value in numbers:
        percent = 100 * value / total
        result.append(percent)
    return result

In [2]:
visits = [15, 35, 80]
percentages = normalize(visits)
print(percentages)

[11.538461538461538, 26.923076923076923, 61.53846153846154]


현실의 데이터는 매우 클 가능성이 높기 때문에 제너레이터를 정의하여 효율적으로 데이터 처리가 가능하다.

In [5]:
def generator_visits(numbers):
    for number in numbers:
        yield number

하지만 제너레이터로 생성한 값을 normalize 함수의 파라미터로 사용하면 결과가 나오지 않는다.

In [6]:
numbers = [15, 35, 80]
it = generator_visits(numbers)
percentages = normalize(it)
print(percentages)

[]


그 이유는 이터레이터가 결과를 한 번만 생성하기 때문이다.<br>
한 번의 순회를 마친 이터레이터는 아무 결과도 출력하지 않는다.

In [7]:
it = generator_visits(numbers)
print(list(it))
print(list(it))

[15, 35, 80]
[]


대부분의 파이썬의 동작에서 처음부터 비어있는 이터레이터와 결과가 있었지만 이미 사용한 이터레이터를 구분하지 않는다.<br>
<br>
이 문제를 해결하기 위해 이터레이터의 복사본을 리스트에 저장하는 방법을 사용할 수 있다.<br>
리스트는 몇 번이고 순회가 가능하다.

In [9]:
def normalize_copy(numbers):
    numbers = list(numbers)
    total = sum(numbers)
    result = []
    for value in numbers:
        percent = 100 * value / total
        result.append(percent)
    return result

In [10]:
numbers = [15, 35, 80]
it = generator_visits(numbers)
percentages = normalize_copy(it)
print(percentages)

[11.538461538461538, 26.923076923076923, 61.53846153846154]


하지만 복사한 리스트가 큰 경우 문제가 발생한다.<br>
메모리 부족으로 인해 올바른 동작을 하지 않을 가능성도 존재한다.<br>
<br>
메모리 부족 문제를 해결하려면 함수의 파라미터로 이터레이터를 반환하는 함수를 설정하는 것이다.

In [12]:
def normalize_func(get_iter):
    total = sum(get_iter())    # 새 이터레이터
    result = []
    for value in get_iter():   # 새 이터레이터
        percent = 100 * value / total
        result.append(percent)
    return result

람다 표현식으로 원하는 결과를 얻을 수 있다.

In [13]:
percentages = normalize_func(lambda : generator_visits(numbers))
print(percentages)

[11.538461538461538, 26.923076923076923, 61.53846153846154]


람다 함수를 직접적으로 인수로 사용하는 것은 세련되지 못 하다.<br>
이터레이터 프로토콜을 구현한 새 컨테이너 클래스를 사용하면 더 세련된 방법으로 같은 결과를 얻을 수 있다.<br>
이터레이터 프로토콜은 컨테이너 타입의 내용을 순서대로 탐색하는 방법을 나타낸다.<br>
```for i in foo```같은 구문에서 python은 ```iter(foo)```를 호출하고, 함수 iter는 특정 메서드인 ```foo.__iter__```를 호출한다.<br>
```__iter__```메서드는 이터레이터 객체를 반환한다.<br>
for 루프는 이터레이터를 모두 소진해 StopIteration 예외가 발생할 때까지 이터레이터 객체의 함수 next를 계속 호출한다.<br>
<br>
```__iter__``` 메서드는 제널이터로 구현하면 더 쉽게 구현이 가능하다.<br>
다음이 우리의 목적에 맞게 구성한 컨테이너 크래스다.

In [14]:
class ReadVisits(object):
    def __init__(self, numbers):
        self.numbers = numbers
    
    def __iter__(self):
        for number in self.numbers:
            yield number

이 클래스를 사용할 경우 맨 처음의 함수를 수정하지 않아도 올바른 결과를 얻을 수 있다.

In [15]:
visits = ReadVisits(numbers)
perecentages = normalize(visits)
print(percentages)

[11.538461538461538, 26.923076923076923, 61.53846153846154]


normalize의 sum 메서드, for 루프 각각 ```ReadVisits.__iter__```를 호출하여 이터레이터 객체를 할당한다.<br>
그리고 두 객체들은 서로 독립적이기 때문에 올바른 동작을 한다.<br>
<br>
normalize 함수의 올바른 동작을 보장하기 위해서 단순 이터레이터를 인수로 받아들이지 않도록 함수를 작성한다.<br>
>내장 함수 ```iter```가 이터레이터를 받으면 입력받은 이터레이터 자체가 반환된다.<br>
>하지만 ```iter```가 컨테이너 타입을 받으면 항상 다른 이터레이터 객체가 반환된다.<br>

이것을 이용해서 이터레이터를 입력으로 받으면 TypeError를 일으키도록 할 수 있다.

In [16]:
def normalize_defensive(numbers):
    if iter(numbers) is iter(numbers):
        raise TypeError('Must supply a conatainer')
    total = sum(numbers)
    result = []
    for value in numbers:
        percent = 100 * value / total
        result.append(percent)
    return result

컨테이너 타입인 리스트와 ReadVisits에 대해서 올바르게 작동하는 것을 확인할 수 있다.

In [17]:
visits = [15, 35, 80]
print(normalize_defensive(visits))
visits = ReadVisits(visits)
print(normalize_defensive(visits))

[11.538461538461538, 26.923076923076923, 61.53846153846154]
[11.538461538461538, 26.923076923076923, 61.53846153846154]


이터레이터에 대해서는 TypeError를 일으킨다.

In [18]:
it = iter(visits)
normalize_defensive(it)

TypeError: Must supply a conatainer